### このノートブックでは自作した簡単なLED回路を読み込み、LEDを光らせるまでの一連の流れを記録している。

Overlay("./led_overlay.bit").download()で回路(bitファイル)を読み込むことが出来る。詳しくは同じディレクトリの”PL.Overlayについて”を参照。

In [1]:
from pynq import Overlay
# load Base Overlay
Overlay("./led_overlay.bit").download()

読み込んだled_overlay.bitは以下のようになっている。

![](led_overlay.png)

上記回路のaxi_gpio_0にアクセスすることでLEDの挙動を変化させることが出来る。作成した時の名前はaxi_gpio_0であったが、tclファイルではSEG_axi_gpio_0_Regへと変わっている。ビットストリームの生成時に回路の名前の先頭にSEG_が追加され、語尾に_Regが追加されるようだ。  

このSEG_axi_gpio_0_Regにアクセスするためには、そのベースアドレスを知る必要がある。PYNQではbitファイルの読み込みとともに、tclファイルの情報を解析し、ip_dictという名前の辞書が作成される。回路名とそのアドレス、オフセット、状態(デフォルトはNone)が一つにまとまっており、回路の名前からアドレスやその他の情報を調べることが出来る。

##### ip_dictについて(PYNQ公式ドキュメントより引用)
>The Overlay package generates a dictionary called ip_dict containing the names of IP in a specific overlay (e.g. base.bit). The dictionary can be used to reference an IP by name in your Python code, rather than by a hard coded address. It can also check the IP available in an overlay.  
...  
>Note, this parses the tcl file that was exported with the bitstream. It does not do check the overlay currently running in the PL.  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html

In [2]:
#ip_dictは辞書。ビットストリームファイルが読み込まれると同時に作成される。
#ある回路の名前とそれに対応するメモリの情報(['0x41200000', '0x00010000', None])がセットになっている。
from pynq import PL
PL.ip_dict["SEG_axi_gpio_0_Reg"]

['0x41200000', '0x00010000', None]

led_overlay.tclにおける1063行目

create_bd_addr_seg -range 0x00010000 -offset 0x41200000 [get_bd_addr_spaces processing_system7_0/Data] [get_bd_addr_segs axi_gpio_0/S_AXI/Reg] SEG_axi_gpio_0_Reg

### LEDを点灯させる(以下のコードではLEDが全て点灯する)

In [3]:
#"xilinx/pynq/mmio.py”と"xilinx/pynq/pl.py"を使っている
#MMIOは/デバイスファイルdev/memを書き換えるための関数

from pynq import MMIO
from pynq import PL

led = MMIO(int(PL.ip_dict["SEG_axi_gpio_0_Reg"][0],16),16)
led.write(0x0,0B1111) #0B(led4)(led3)(led2)(led1)

### 参考
Document  
http://pynq.readthedocs.io/en/latest/16_creating_overlays.html?highlight=tcl　　
Overlayとpythonのやり取りについて網羅しているサイト。  
http://todotani.cocolog-nifty.com/blog/2017/01/pynq-z1overlayp.html  
xilinx/pynq/pl.pyのコード  
http://pynq.readthedocs.io/en/latest/_modules/pynq/pl.html  